# Processes

- Data producer and consumer  running at different speed
- Communication via queues

In [ ]:
import multiprocessing as mp
import queue
import time
import random
import os

# Data producer worker

In [ ]:
def producer (args):
    prefix, maxN = args
    global dataQueue    
    print ("Started producer ---->", os.getpid())
    for i in range(maxN):
        data = "%s-%d" % (prefix, i)        
        dataQueue.put(data, block=True)    
        time.sleep(random.random() * 0.1)

# Data consumer worker

In [ ]:
def consumer (label):
    global dataQueue
    print ("Started consumer", os.getpid())
    while True:
        try:
            data = dataQueue.get(block=True, timeout=1)
            print ("Consumer %*d" % ((1+label), label), data)
            time.sleep(random.random()*0.5)
        except queue.Empty:
            print ("Emtry")
            break
    print ("End")

# Example: One producer and one consumer

In [ ]:
# Create queue
dataQueue = mp.Queue(maxsize=10)

# Create the processes

consumerProc = mp.Process(target=consumer, args=(0,))
consumerProc.start()

print ("Start producing")
producer (('A', 10))
print ("End producing")

consumerProc.join()

# Close the queue
dataQueue.close()

# Example: One producer and multiple consumers

In [ ]:
dataQueue = mp.Queue(maxsize=4)

[ mp.Process(target=consumer, args=(i,)).start() for i in range(5) ]

producer (('A', 30))

dataQueue.close()


# Example: Multiple producers and multiple consumers

In [ ]:
dataQueue = mp.Queue(maxsize=4)

[ mp.Process(target=consumer, args=(i,)).start() for i in range(5) ]

[ mp.Process(target=producer, args=((chr(i+ord('A')),(1+i)*10),)).start() for i in range(3) ]

dataQueue.close()